# Automated Customer Support using AWS SageMaker

This notebook demonstrates two approaches for automating customer support queries:

1. **RoBERTa for Complex Queries:**
   - Leverages AWS SageMaker for scalable, production-grade inference.
   - Ideal for complex customer queries requiring deep contextual understanding (using 5 intent classes).

2. **DistilBERT for Fast, Simple Queries:**
   - Performs local inference using DistilBERT, optimized for low latency.
   - Suitable for straightforward queries like checking account balances or order statuses (using 3 intent classes).

Detailed comments and explanations are provided in each cell.

In [ ]:
# Import required libraries
import boto3              # AWS SDK for Python to interact with AWS services
import json               # To convert Python objects to JSON format
from transformers import RobertaTokenizer, RobertaForSequenceClassification  # RoBERTa for intent classification
import torch              # PyTorch for tensor operations

# Load the pre-trained RoBERTa tokenizer and model for intent classification
# 'num_labels=5' indicates five different customer support intents (e.g., order change, refund, inquiry, etc.)
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=5)

# Define example customer queries for complex query processing
queries = [
    "How can I change my order?",
    "I need help with a refund."
]

# Tokenize the input queries
# - padding=True: Pads queries to ensure uniform length
# - truncation=True: Truncates queries exceeding the maximum allowed length
# - return_tensors="pt": Returns the results as PyTorch tensors
inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")

# Convert tensor of input IDs to a list and then to a JSON formatted payload
payload = json.dumps({"input_ids": inputs['input_ids'].tolist()})

# Initialize the SageMaker runtime client
sagemaker_runtime = boto3.client("runtime.sagemaker")

# Specify the SageMaker endpoint name (ensure this matches your deployed endpoint)
endpoint_name = "customer-support-endpoint"

try:
    # Invoke the SageMaker endpoint with the JSON payload
    response = sagemaker_runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        ContentType="application/json",
        Body=payload
    )

    # Read and decode the response from the endpoint
    result = response['Body'].read().decode('utf-8')
    print("SageMaker Endpoint Response:", result)
except Exception as e:
    # Print any errors that occur during invocation
    print("Error invoking SageMaker endpoint:", str(e))

### DistilBERT for Fast, Simple Customer Queries

In this section, we perform local inference using DistilBERT. This model is optimized for speed and lower resource usage, making it ideal for simple queries like checking account balances or order statuses. We use three intent classes (for example: 0 for Account Balance, 1 for Order Status, and 2 for Product Info).

In [ ]:
# Import the DistilBERT tokenizer and model for intent classification
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load the pre-trained DistilBERT tokenizer and model
# 'num_labels=3' sets up classification for three types of intents
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=3)

# Define example customer queries for simple query processing
queries = [
    "What is my account balance?",
    "Where is my order?"
]

# Tokenize the queries
inputs = tokenizer(queries, padding=True, truncation=True, return_tensors="pt")

# Run inference without gradient calculation to speed up the process
with torch.no_grad():
    outputs = model(**inputs)
    # The model outputs raw logits; we take the argmax to select the predicted intent class
    predictions = torch.argmax(outputs.logits, dim=-1)

# Print the sentiment predictions
# Convention: 0 indicates Account Balance, 1 indicates Order Status, 2 indicates Product Info
print("DistilBERT Intent Predictions:", predictions.tolist())

### Conclusion

This notebook demonstrated two methods for automating customer support queries:

1. **RoBERTa on AWS SageMaker:**
   - Suited for handling complex customer queries that require deep contextual understanding.
   - Integrated with AWS services for scalable inference (using a SageMaker endpoint).

2. **DistilBERT for Local Inference:**
   - Optimized for fast, low-latency inference for simpler queries.
   - Runs directly on local resources, making it suitable for on-device applications.

These approaches can be integrated with AWS Lex or AWS Lambda to build comprehensive automated customer support systems.